In [4]:
#Gauss elimination with partial pivoting, LU 
#Decomposition, Jacobi and Gauss-Seidel Methods

import numpy as np

class LinearSystemSolver:
    def __init__(self, A, b):
        self.A = np.array(A, dtype=float)
        self.b = np.array(b, dtype=float)
        self.n = len(b)

    # ---------------- Gauss Elimination with Partial Pivoting ----------------
    def gauss_elimination(self):
        A = self.A.copy()
        b = self.b.copy()
        n = self.n

        for k in range(n - 1):
            # Partial pivoting
            max_row = np.argmax(abs(A[k:, k])) + k
            if A[max_row, k] == 0:
                raise ValueError("Matrix is singular!")
            if max_row != k:
                A[[k, max_row]] = A[[max_row, k]]
                b[[k, max_row]] = b[[max_row, k]]

            for i in range(k + 1, n):
                factor = A[i, k] / A[k, k]
                A[i, k:] -= factor * A[k, k:]
                b[i] -= factor * b[k]

        # Back substitution
        x = np.zeros(n)
        for i in range(n - 1, -1, -1):
            x[i] = (b[i] - np.dot(A[i, i + 1:], x[i + 1:])) / A[i, i]
        return x

    # ---------------- LU Decomposition (Doolittle's method) ----------------
    def lu_decomposition(self):
        A = self.A.copy()
        n = self.n
        L = np.zeros((n, n))
        U = np.zeros((n, n))

        for i in range(n):
            # Upper Triangular
            for k in range(i, n):
                U[i, k] = A[i, k] - sum(L[i, j] * U[j, k] for j in range(i))

            # Lower Triangular
            L[i, i] = 1
            for k in range(i + 1, n):
                L[k, i] = (A[k, i] - sum(L[k, j] * U[j, i] for j in range(i))) / U[i, i]

        return L, U

    def solve_lu(self):
        L, U = self.lu_decomposition()
        b = self.b.copy()
        n = self.n

        # Forward substitution (Ly = b)
        y = np.zeros(n)
        for i in range(n):
            y[i] = b[i] - np.dot(L[i, :i], y[:i])

        # Back substitution (Ux = y)
        x = np.zeros(n)
        for i in range(n - 1, -1, -1):
            x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        return x

    # ---------------- Jacobi Iterative Method ----------------
    def jacobi(self, tol=1e-10, max_iterations=500):
        A = self.A.copy()
        b = self.b.copy()
        n = self.n
        x = np.zeros(n)

        for _ in range(max_iterations):
            x_new = np.zeros_like(x)
            for i in range(n):
                s = sum(A[i][j] * x[j] for j in range(n) if j != i)
                x_new[i] = (b[i] - s) / A[i][i]
            if np.linalg.norm(x_new - x, ord=np.inf) < tol:
                return x_new
            x = x_new
        return x

    # ---------------- Gauss-Seidel Iterative Method ----------------
    def gauss_seidel(self, tol=1e-10, max_iterations=500):
        A = self.A.copy()
        b = self.b.copy()
        n = self.n
        x = np.zeros(n)

        for _ in range(max_iterations):
            x_new = np.copy(x)
            for i in range(n):
                s1 = sum(A[i][j] * x_new[j] for j in range(i))
                s2 = sum(A[i][j] * x[j] for j in range(i + 1, n))
                x_new[i] = (b[i] - s1 - s2) / A[i][i]
            if np.linalg.norm(x_new - x, ord=np.inf) < tol:
                return x_new
            x = x_new
        return x


# ---------------- Example Usage ----------------
if __name__ == "__main__":
   A = [[10, -1, 2, 0],
     [-1, 11, -1, 3],
     [2, -1, 10, -1],
     [0, 3, -1, 8]]
   b = [6, 25, -11, 15]

solver = LinearSystemSolver(A, b)

print("Gauss Elimination:", solver.gauss_elimination())
print("LU Decomposition:", solver.solve_lu())
print("Jacobi Method:", solver.jacobi())
print("Gauss-Seidel Method:", solver.gauss_seidel())


Gauss Elimination: [ 1.  2. -1.  1.]
LU Decomposition: [ 1.  2. -1.  1.]
Jacobi Method: [ 1.  2. -1.  1.]
Gauss-Seidel Method: [ 1.  2. -1.  1.]
